In [1]:
import pandas as pd
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import time
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from collections import Counter
from IPython.display import clear_output

from scripts import BpeTokenizer, Model, Trainer, Collator, MyDataset, generate

# Загружаем данные

In [2]:
df = pd.read_csv('data/dataset.csv')
train_texts = df['text'][:-1024].tolist()
eval_texts = df['text'][-1024:].tolist()

# Инициализируем и обучаем токенизатор

In [3]:
tokenizer = BpeTokenizer()

In [4]:
tokenizer.train(train_texts[:2048], max_vocab=2048)

pair=(277, 338), freq=52: 100%|██████████| 1789/1789 [02:34<00:00, 11.55it/s]  


# Создаем датасеты и Collator

In [5]:
train_dataset = MyDataset(train_texts, tokenizer, max_length=128)
eval_dataset = MyDataset(eval_texts, tokenizer, max_length=128)
collator = Collator(tokenizer.pad_token_id)

100%|██████████| 1024/1024 [00:15<00:00, 64.79it/s]


# Создаем модель

In [6]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [16]:
model = Model(tokenizer.get_vocab_size(), emb_size=128, hidden_size=256, num_layers=3, dropout=0.2)

# Создаем Trainer и запускаем обучение

In [8]:
train_loader = DataLoader(
            train_dataset,
            batch_size=32,
            shuffle=True,
            drop_last=True,
            collate_fn=collator
        )

In [9]:
# for batch in train_loader:
#     print(batch.shape)
#     print(batch)
#     x = batch[:, :-1]
#     y = batch[:, 1:]
#     print(x.shape)
#     print(y.shape)

#     logits, _ = model(x)
#     print(logits.reshape(-1, logits.size(-1)).shape)
#     print(y.reshape(-1).shape)
#     break

In [17]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    n_epochs=12,
    train_batch_size=32,
    eval_batch_size=32,
    eval_steps=64,
    collator=collator,
    lr=2e-2,
    ignore_index=tokenizer.pad_token_id
)

In [18]:
trainer.train()

epoch=0.126953125, loss=5.702388286590576:   1%|          | 65/6144 [00:18<1:56:29,  1.15s/it]

epoch=0.125, eval_loss=5.583472698926926


epoch=0.251953125, loss=4.881402492523193:   2%|▏         | 129/6144 [00:37<1:56:07,  1.16s/it]

epoch=0.25, eval_loss=4.758529752492905


epoch=0.376953125, loss=4.40933084487915:   3%|▎         | 193/6144 [00:56<1:54:25,  1.15s/it] 

...

epoch=11.75, eval_loss=3.107367791235447


epoch=11.875, loss=3.2180399894714355:  99%|█████████▉| 6080/6144 [2:08:03<01:36,  1.50s/it]      

epoch=11.875, eval_loss=3.104408249258995


epoch=12.0, loss=3.198725700378418: 100%|██████████| 6144/6144 [2:10:22<00:00,  1.27s/it]         

epoch=12.0, eval_loss=3.1031976342201233


In [19]:
torch.save(model.state_dict(), './lstm.pt')

# Оцениваем качество и проверяем жадную и случайную генерацию

In [20]:
trainer.evaluate()

3.1031976342201233

In [21]:
generate(model, tokenizer, temperature=0)

'Если вы не стоит учесть вам стандартов. В это время вы можете столкнуться в делах с близким и любимыми. В это время вы можете столкнуться в делах с близким и любимыми. В это время вы можете оказаться вам стандартов вам станет более правильно столкноветься в такой обстановке. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам представителям знака не стоит вам не стоит вам не стать вами. В это время вы можете оказаться вам п

In [22]:
generate(model, tokenizer, temperature=0.5, top_k=20)

'Сегодня звезды советуют Стрельцам не стоит вам не стать вам старые знакомства, велика. Например, работы, например, моральных и профессиональных), мелких и финансовых действий. Не стоит предупреждиться в конце дня ответственных, так как вы можете оказаться вам стоит быть вам поставиться вам производственных, правилах», закончится о себе. В этот день вы будете деловатым и надеями, деловыми партнерами, стремитесь какое-то другие навыки в течение дня и надежно и недостаточно старшимися вопросами. Не стоит отказываться от представителей знака преодолеть недовольство, обновательства, отдельности в недвижимости, ориентирует вам, вы увлекаете, какое-то старые знакомства. Например, так как вам будет не стоит заниматься зависимыми надеждами. Тем больше свободно отказаться на вами или над ошибки, деловых людей. В этот день будет дать вам представителям знака непредсказуемым вам правильно и контролировать какую-то вами. Вероятно, если вы будете не потребоваться в несмотря на будущего. В это время